## Word to Vector

Word2vec (word embedding)把词用向量表示。

有Skip-gram 和CBOW 两种训练方法，Skip-gram 基于中心词生成背景词，CBOW 基于背景词生成中心词。数据量大时，Skip-gram效果好。

Skip-gram 有两种近似训练方式，负采样和层序Softmax，本文只讨论负采样。

Word2vec被提出后，很多其他词嵌入模型也陆续发表，有代表性的斯坦福团队发表的GloVe。


### Skip-gram

Skip-gram 思路是相邻越近的两个词的相似度越高。越远的相似度越低。

- 近义词：两个词是近义词，则这两个词分别邻近的词会很相似，所以近义词的词向量会很相似。

- 二次采样：对于出现频率高的词，比如介词the，所有的名词都可能跟the 邻近，但这些名词词向量肯定大同小异，因此去掉高频词，训练出的词向量效果更好。论文中有采样公式。

- 负采样：求解中心词出现的条件下背景词出现的概率，需要两个词向量的内积，除以中心词和其他所有词的内积之和，并做Softmax激活。这样feed propagation，back propagation都要计算 $O$(词典大小) 时间复杂度。引入负样本的思想，以窗口为界，中心词和窗口内词为正样本，中心词和窗口外词为负样本，只要计算和求导只要更新少量词的向量即可。

  负采样改变了目标函数，正、负样本分别做词向量内积，用Sigmoid激活之后求积。有点类似Logistic Regression里面交叉熵损失函数，去掉指数后的乘积。

    1. trick：按窗口大小随机取s = [1, window_size]，正样本一次取s个，让模型更关注距离中心词更近的词。
    2. 负样本选取：词出现频数的0.75 次幂作为权重，一个正样本，对词典随机选取K个负样本。减弱频次差异的影响，让小概率词被采样概率变大。
    

#### Skip-gram推导

##### Skip-gram最大化似然函数中条件概率表示成Softmax

每个词被表示成两个$d$维向量，词在词典中索引为$i$，当它为中心词时向量表示为$\boldsymbol{v}_i\in\mathbb{R}^d$，而为背景词时向量表示为$\boldsymbol{u}_i\in\mathbb{R}^d$。词典索引集$\mathcal{V} = \{0, 1, \ldots, |\mathcal{V}|-1\}$。

设中心词$w_c$在词典中索引为$c$，背景词$w_o$在词典中索引为$o$，给定中心词生成背景词的条件概率：
$$P(w_o \mid w_c) = \frac{\text{exp}(\boldsymbol{u}_o^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)} \tag{1}$$

给定一个长度为$T$的文本序列，设时间步$t$的词为$w^{(t)}$。假设给定中心词的情况下背景词的生成相互独立，当背景窗口大小为$m$时，最大似然函数即给定任一中心词生成所有背景词的概率：
$$ \prod_{t=1}^{T} \prod_{-m \leq j \leq m,\ j \neq 0} P(w^{(t+j)} \mid w^{(t)}) \tag{2}$$

等价于最小化损失函数：
$$ - \sum_{t=1}^{T} \sum_{-m \leq j \leq m,\ j \neq 0} \text{log}\, P(w^{(t+j)} \mid w^{(t)})$$

随机梯度下降，每次迭代随机采样较短子序列，计算该子序列的损失，然后计算梯度更新模型参数。梯度计算的关键是，条件概率的对数，对中心词向量和背景词向量的梯度。
$$\log P(w_o \mid w_c) =
\boldsymbol{u}_o^\top \boldsymbol{v}_c - \log\left(\sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)\right)$$

$$\begin{aligned}
\frac{\partial \text{log}\, P(w_o \mid w_c)}{\partial \boldsymbol{v}_c} 
&= \boldsymbol{u}_o - \frac{\sum_{j \in \mathcal{V}} \exp(\boldsymbol{u}_j^\top \boldsymbol{v}_c)\boldsymbol{u}_j}{\sum_{i \in \mathcal{V}} \exp(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}\\
&= \boldsymbol{u}_o - \sum_{j \in \mathcal{V}} \left(\frac{\text{exp}(\boldsymbol{u}_j^\top \boldsymbol{v}_c)}{ \sum_{i \in \mathcal{V}} \text{exp}(\boldsymbol{u}_i^\top \boldsymbol{v}_c)}\right) \boldsymbol{u}_j\\ 
&= \boldsymbol{u}_o - \sum_{j \in \mathcal{V}} P(w_j \mid w_c) \boldsymbol{u}_j
\end{aligned}
$$

训练结束后，对于词典中的任一索引为$i$的词，我们均得到该词作为中心词和背景词的两组词向量$\boldsymbol{v}_i$和$\boldsymbol{u}_i$。

对中心词$w_c$，计算$\boldsymbol{v}_c$的梯度，需要词典中所有以$w_c$为中心词的条件概率，每一步梯度计算时间复杂度是$\boldsymbol{O}(|\mathcal{V}|)$，引入负采样解决此问题。

##### 负采样近似

设背景词$w_o$出现在中心词$w_c$的背景窗口为事件$P$，分布$P(w)^{0.75}$作为权重，采样$K$个未出现在该背景窗口中的噪声词。噪声词$w_k$（$k=1, \ldots, K$）不出现在中心词$w_c$的该背景窗口为事件$N_k$。假设含有正样本和负样本的事件$P, N_1, \ldots, N_K$相互独立，负采样最大化联合概率与式(2)相同。条件概率跟式(1)不同，近似表示为：
$$ P(w^{(t+j)} \mid w^{(t)}) = P(D=1\mid w^{(t)}, w^{(t+j)})\prod_{k=1,\ w_k \sim P(w)}^K P(D=0\mid w^{(t)}, w_k).$$

$D = 1$ 表示正样本，$D = 0$ 表示负样本。

对事件$P$的概率，用Sigmoid激活，
$$P(D=1\mid w_c, w_o) = \sigma(\boldsymbol{u}_o^\top \boldsymbol{v}_c) = \frac{1}{1 + \exp(-\boldsymbol{u}_o^\top \boldsymbol{v}_c)}$$

对事件$N$的概率，
$$P(D=0\mid w_c, w_o) = 1 - P(D=1\mid w_c, w_o) = 1 - \sigma(\boldsymbol{u}_o^\top \boldsymbol{v}_c) = \frac{1}{1 + \exp(\boldsymbol{u}_o^\top \boldsymbol{v}_c)}$$

设文本序列中时间步$t$的词$w^{(t)}$在词典中的索引为$i_t$，噪声词$w_k$在词典中的索引为$h_k$。条件概率的对数损失为：

$$\begin{aligned}
\log P(w^{(t+j)} \mid w^{(t)})
=& \log P(D=1\mid w^{(t)}, w^{(t+j)}) + \sum_{k=1,\ w_k \sim P(w)}^K \log P(D=0\mid w^{(t)}, w_k)\\
=& \log\, \sigma\left(\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right) + \sum_{k=1,\ w_k \sim P(w)}^K \log\left(1-\sigma\left(\boldsymbol{u}_{h_k}^\top \boldsymbol{v}_{i_t}\right)\right)\\
=& \log\, \sigma\left(\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right) + \sum_{k=1,\ w_k \sim P(w)}^K \log\sigma\left(-\boldsymbol{u}_{h_k}^\top \boldsymbol{v}_{i_t}\right)
\end{aligned}
$$

$$ \because \sigma'(z) = \sigma(z) \, (1 - \sigma(z)) = \sigma(z) \, \sigma(-z)$$

$$\begin{aligned}
\therefore \frac{\partial \log P(w^{(t+j)} \mid w^{(t)})}{\partial v_{i_t}}
=& \frac{\sigma\left(\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right) \sigma\left(-\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right) \boldsymbol{u}_{i_{t+j}}^\top}{\sigma\left(\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right)} + \sum_{k=1,\ w_k \sim P(w)}^K \frac{\sigma\left(-\boldsymbol{u}_{h_k}^\top \boldsymbol{v}_{i_t}\right) \sigma\left(\boldsymbol{u}_{h_k}^\top \boldsymbol{v}_{i_t}\right) (-\boldsymbol{u}_{h_k}^\top)}{\sigma\left(-\boldsymbol{u}_{h_k}^\top \boldsymbol{v}_{i_t}\right)} \\
=& \sigma\left(-\boldsymbol{u}_{i_{t+j}}^\top \boldsymbol{v}_{i_t}\right) \boldsymbol{u}_{i_{t+j}}^\top - \sum_{k=1,\ w_k \sim P(w)}^K \sigma\left(\boldsymbol{u}_{h_k}^\top \boldsymbol{v}_{i_t}\right) \boldsymbol{u}_{h_k}^\top \\
=& P(D=0\mid w^{(t)}, w^{(t+j)}) \boldsymbol{u}_{i_{t+j}}^\top - \sum_{k=1,\ w_k \sim P(w)}^K P(D=1\mid w^{(t)}, w_k) \boldsymbol{u}_{h_k}^\top
\end{aligned}
$$

训练中每步的梯度计算开销不再与词典大小相关，而与$K$线性相关。当$K$取较小的常数时，负采样在每一步的梯度计算开销较小。

##### 中心词向量和背景词向量的区别
在自然语言处理应用中，一般使用中心词向量作为词的表征向量。

给定序列$abcd$，window_size = 1，最大似然函数是三对相互生成的条件概率。
$$P(b|a) P(a|b) P(c|b) P(b|c) P(d|c) P(c|d)$$

$$P(b|a) P(a|b) = \frac{\text{exp}(u_b^\top v_a)}{\sum_i \text{exp}(u_i^\top v_a)} \frac{\text{exp}(u_a^\top v_b)}{\sum_i \text{exp}(u_i^\top v_b)}$$

把$u$ 和$v$ 向量互换，分子变成$\text{exp}(v_b^\top u_a) \, \text{exp}(v_a^\top u_b)$，不影响全概率分子大小，但分母上稍有差别。

#### Skip-gram Code
[Github code](https://github.com/LaoLiulaoliu/tensorflow_vs_mxnet_cpu/blob/main/skipgram.py)

### GloVe
使用了词与词之间的共现（co-occurrence）信息。我们定义$\boldsymbol{X}$ 为共现词频矩阵，其中元素$x_{ij}$为词$w_j$出现在词$w_i$的背景的次数。例如，在一段文本序列中，如果词$w_j$出现在词$w_i$左边或者右边不超过10个词的距离，认为词$w_j$出现在词$w_i$的背景一次。令$x_i = \sum_k x_{ik}$为任意词出现在词$w_i$的背景的次数。那么，数据集中以$w_i$为中心词生成背景词$w_j$的共现概率

$$p_{ij} = \mathbb{P}(w_j \mid w_i) = \frac{x_{ij}}{x_i}$$

为词$j$在词$i$的背景中出现的条件概率。这一条件概率也称词$i$和词$j$的共现概率。


#### 用词向量表达共现概率比值
GloVe的核心思想在于使用词向量表达共现概率比值。

GloVe论文里展示了词对以ice（冰）和steam（蒸汽）为中心词的共现概率以及它们之间的比值 [3]：

* $\mathbb{P}(k \mid \text{ice})$：0.00019（$k$= solid），0.000066（$k$= gas），0.003（$k$= water），0.000017（$k$= fashion）
* $\mathbb{P}(k \mid \text{steam})$：0.000022（$k$= solid），0.00078（$k$= gas），0.0022（$k$= water），0.000018（$k$= fashion）
* $\mathbb{P}(k \mid \text{ice}) / \mathbb{P}(k \mid \text{steam})$：8.9（$k$= solid），0.085（$k$= gas），1.36（$k$= water），0.96（$k$= fashion）

观察到以下现象：

* 对于与ice相关而与steam不相关的词$k$，例如$k=$solid（固体），我们期望共现概率比值$p_{ik}/p_{jk}$较大，例如上面最后一行结果中的值8.9。
* 对于与ice不相关而与steam相关的词$k$，例如$k=$gas（气体），我们期望共现概率比值$p_{ik}/p_{jk}$较小，例如上面最后一行结果中的值0.085。
* 对于与ice和steam都相关的词$k$，例如$k=$water（水），我们期望共现概率比值$p_{ik}/p_{jk}$接近1，例如上面最后一行结果中的值1.36。
* 对于与ice和steam都不相关的词$k$，例如$k=$fashion（时尚），我们期望共现概率比值$p_{ik}/p_{jk}$接近1，例如上面最后一行结果中的值0.96。

由此可见，共现概率比值能比较直观地表达词与词之间的关系。可以构造一个词向量函数，来表达共现概率比值。而任意一个这样的比值需要三个词$w_i$、$w_j$和$w_k$的词向量。以$w_i$作为中心词的共现概率比值为${p_{ij}}/{p_{ik}}$。$\boldsymbol{v}_i$和${\boldsymbol{u}}_i$分别表示词$w_i$作为中心词和背景词的词向量。

我们可以用有关词向量的函数$f$来表达共现概率比值：

$$f(\boldsymbol{u}_j, \boldsymbol{u}_k, {\boldsymbol{v}}_i) \approx \frac{p_{ij}}{p_{ik}}.$$

函数$f$可能的设计并不唯一，我们只需考虑一种较为合理的可能性。

首先，用向量之差来表达共现概率的比值，并将上式改写成

$$f(\boldsymbol{u}_j - \boldsymbol{u}_k, {\boldsymbol{v}}_i) \approx \frac{p_{ij}}{p_{ik}}.$$

共现概率比值是一个标量，将$f$限制为一个标量函数，可以使用向量之间的内积把函数$f$的自变量进一步变为，

$$f((\boldsymbol{u}_j - \boldsymbol{u}_k)^\top {\boldsymbol{v}}_i) = \frac{f(\boldsymbol{u}_j^\top {\boldsymbol{v}}_i)}{f(\boldsymbol{u}_k^\top {\boldsymbol{v}}_i)} \approx \frac{p_{ij}}{p_{ik}}.$$

交换索引$j$和$k$后可以看到函数$f$左右式应该满足$f(x)f(-x) \approx 1$，因此一种可能是$f(x)=\exp(x)$，

$$f((\boldsymbol{u}_j - \boldsymbol{u}_k)^\top {\boldsymbol{v}}_i) = \frac{\exp(\boldsymbol{u}_j^\top {\boldsymbol{v}}_i)}{\exp(\boldsymbol{u}_k^\top {\boldsymbol{v}}_i)} \approx \frac{p_{ij}}{p_{ik}}.$$

满足最右边约等号的一种可能是$\exp\left(\boldsymbol{u}_j^\top {\boldsymbol{v}}_i\right) \approx \alpha p_{ij}$，($\alpha$是常数)。考虑到$p_{ij}=x_{ij}/x_i$，取对数后$\boldsymbol{u}_j^\top {\boldsymbol{v}}_i \approx \log\,\alpha + \log\,x_{ij} - \log\,x_i$。使用中心词偏差项$b_i$和背景词偏差项$c_j$来拟合$\log\,x_i$和$-\log\,\alpha$，词向量表达共现词频的对数：

$$\boldsymbol{u}_j^\top \boldsymbol{v}_i + b_i + c_j \approx \log(x_{ij}).$$

将索引$i$和$j$互换，约等式成立，可见，任意一对词共现的对称性两性质：

* 任意词作为中心词和背景词的词向量应该相等：对任意词$w_i$，$\boldsymbol{v}_i = {\boldsymbol{u}}_i$。
* 词与词之间共现词频矩阵$\boldsymbol{X}$ 应该对称：对任意词$w_i$和$w_j$，$x_{ij} = x_{ji}$。

#### 损失函数

GloVe中的共现词频是直接在训练数据上统计得到的。为了学习词向量和相应的偏差项，我们希望上式中的左边与右边尽可能接近，上式左右两边取平方误差并加权，得到GloVe的损失函数。给定词典$\mathcal{V}$和权重函数$h(x_{ij})$，

$$\sum_{i\in\mathcal{V}} \sum_{j\in\mathcal{V}} h(x_{ij}) \left(\boldsymbol{u}_j^\top \boldsymbol{v}_i + b_i + c_j - \log(x_{ij})\right)^2.$$

权重函数$h(x)$是值域在$[0, 1]$的单调递增函数，当$x < c$（例如$c = 100$），令$h(x) = (x/c)^\alpha$（例如$\alpha = 0.75$），反之令$h(x) = 1$。因为$h(0)=0$，所以对于$x_{ij}=0$的平方损失项可以直接忽略。由于权重函数的存在，损失函数的计算复杂度与共现词频矩阵$\boldsymbol{X}$ 中非零元素的数目呈正相关。从$\boldsymbol{X}$ 中随机采样小批量非零元素，并使用优化算法迭代共现词频相关词的向量和偏差项。非零元素$x_{ij}$是预先基于整个数据集计算得到的，包含了数据集的全局统计信息。因此，GloVe模型的命名取“全局向量”（Global Vectors）之意。

如果词$w_i$出现在词$w_j$的背景窗口里，那么词$w_j$也会出现在词$w_i$的背景窗口里，即$x_{ij}=x_{ji}$。不同于word2vec中拟合的是非对称的条件概率$p_{ij}$，GloVe模型拟合的是对称的$log(x_{ij})$。因此，任意词的中心词向量和背景词向量在GloVe模型中是等价的，但由于初始化值的不同，同一个词最终学习到的两组词向量可能不同。当学习得到所有词向量以后，GloVe使用一个词的中心词向量与背景词向量之和作为该词的最终词向量。

### References
[1] [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/pdf/1310.4546.pdf)

[2] [Dive into Deep Learning](http://en.d2l.ai/chapter_natural-language-processing-pretraining/word2vec.html)

[3] Pennington, J., Socher, R., & Manning, C. (2014). Glove: Global vectors for word representation. In Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP) (pp. 1532-1543).